In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sklearn.metrics import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from PIL import Image
import optuna

In [ ]:
BATCHSIZE = 64
EPOCHS = 100
device = torch.device("cuda")
device

In [ ]:
train_data = torchvision.datasets.CIFAR10("./dataset/train/", train=True, download=False, transform=ToTensor())
test_data = torchvision.datasets.CIFAR10("./dataset/test/", train=False, download=False, transform=ToTensor())

train_loader = DataLoader(train_data, batch_size=BATCHSIZE, shuffle=True)
test_loader = DataLoader(test_data, batch_size=BATCHSIZE)

In [ ]:
labels_map = {
    0:'airplane',
    1:'automobile',
    2:'bird',
    3:'cat',
    4:'deer',
    5:'dog',
    6:'frog',
    7:'horse',
    8:'ship',
    9:'truck'
}
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(train_data), size=(1,)).item()
    img, label = train_data[sample_idx]
    pil = torchvision.transforms.ToPILImage()
    img = pil(img)
    figure.add_subplot(rows, cols, i)
    plt.title(labels_map[label])
    plt.axis("off")
    plt.imshow(img)
plt.show()

In [ ]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=5, kernel_size=3)
    self.conv2 = nn.Conv2d(in_channels=5, out_channels=3, kernel_size=3)
    self.conv3 = nn.Conv2d(in_channels=3, out_channels=1, kernel_size=3)
    self.fc1 = nn.Linear(26*26, 50)
    self.fc2 = nn.Linear(50, 10)
  
  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = F.relu(self.conv2(x))
    x = F.relu(self.conv3(x))
    x = x.view(-1, 26*26)
    x = F.relu(self.fc1(x))
    x = F.softmax(self.fc2(x), dim=1)
    return x
model = CNN().to(device)

# optimizer tuning

In [ ]:
def objective(trial: optuna.Trial):
    momentum = trial.suggest_float("momentum", 0.9, 0.999)
    lr = trial.suggest_float("lr", 1e-6, 1e-2)
    alpha = trial.suggest_float("alpha", 0.9, 0.999)
    eps = trial.suggest_float("eps", 1e-8, 1e-4)
    
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters(), lr, momentum, alpha, eps)
    for epoch in range(EPOCHS):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.view(-1, 3*32*32).to(device), target.to(device)
            
            optimizer.zero_grad()
            output = model(data)
            loss = loss_fn(output, target)
            loss.backward()
            optimizer.step()
        
        model.eval()
        correct = 0
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(test_loader):
                data, target = data.view(-1, 32*32*3).to(device), target.to(device)
                output = model(data)
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()
                
        accuracy = correct / len(test_loader.dataset)
        return accuracy
    
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# loss_function tuning

In [ ]:
def objective(trial: optuna.Trial):
    loss_trial = trial.suggest_categorical('loss_fn',['KLDivLoss', 'HuberLoss', 'L1Loss', 'SmoothL1Loss'])
    loss_fn = getattr(nn, loss_trial)()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    for epoch in range(EPOCHS):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)

            optimizer.zero_grad()
            output = model(data)
            out, _ = torch.max(output, dim=1)
            loss = loss_fn(out, target.float())
            loss.backward()
            optimizer.step()
        return loss

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)

In [ ]:
result_lossfn = study.trials_dataframe()
#result_lossfn.to_csv("./optuna_lossfn.csv")
result_lossfn.head()